In [13]:
import matplotlib
matplotlib.rcParams.update({'font.size': 16})
import numpy as np
import matplotlib.pyplot as plt
import csv
from tifffile import imread
import matplotlib.gridspec as gridspec
import cv2
import os
import glob
from sklearn.cluster import KMeans
from matplotlib.colors import LinearSegmentedColormap
import peakutils
from skimage import exposure

Define a colormap for heatmap plots.

In [2]:
colormap = matplotlib.cm.get_cmap('plasma')

Set min & max of z-score plot.

In [3]:
vmin = -2
vmax = 3

Set the frame offset (the number of frames that were cut off from the start of the calcium imaging analysis video).

In [4]:
frame_offset = 0

Set the FPS of tail trace and calcium data.

In [5]:
fps_tail     = 349
fps_calcium  = 3   # frames per second, for one slice

Define a function to be used for blending ROI overlays together to make a final ROI image.

In [6]:
def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image    
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

Define a function for calculating tail beat frequency.

In [7]:
def calculate_tail_beat_frequency(fps, tail_angle_array):
    tail_angles = tail_angle_array.copy()

    baseline = np.mean(tail_angles[:100])
    tail_angles -= baseline

    N = 10
    smoothed_tail_angles = np.convolve(tail_angles, np.ones((N,))/N, mode='same')

    derivative = np.abs(np.diff(smoothed_tail_angles, append=[0]))/0.01
    smoothed_derivative = np.convolve(derivative, np.ones((N,))/N, mode='same')

    threshold = 2
    min_dist = 5
    min_deriv = 10
    highs = peakutils.peak.indexes(smoothed_tail_angles, thres=threshold/max(smoothed_tail_angles), min_dist=min_dist)
    highs = np.array([ h for h in highs if smoothed_derivative[h] > min_deriv ])

    lows = peakutils.peak.indexes(-smoothed_tail_angles, thres=threshold/max(-smoothed_tail_angles), min_dist=min_dist)
    lows = np.array([ h for h in lows if smoothed_derivative[h] > min_deriv ])

    low_freqs = [ 1.0/(lows[i] - lows[i-1]) for i in range(1, len(lows)) ]

    low_freqs_array = np.zeros(smoothed_tail_angles.shape)
    for i in range(len(low_freqs)):
        low_freqs_array[lows[i]:lows[i+1]] = low_freqs[i]

    high_freqs = [ 1.0/(highs[i] - highs[i-1]) for i in range(1, len(highs)) ]

    high_freqs_array = np.zeros(smoothed_tail_angles.shape)
    for i in range(len(high_freqs)):
        high_freqs_array[highs[i]:highs[i+1]] = high_freqs[i]

    freqs_array = (low_freqs_array + high_freqs_array)/2

    return fps*freqs_array

Define a function to create and save the plot given paths to the video, ROI data and tail trace.

In [16]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

def adjust_contrast(image, contrast):
    return image*contrast

def adjust_gamma(image, gamma):
    return exposure.adjust_gamma(image, gamma)

def create_plot(video_path, roi_path, tail_path, **kwargs):
    # define colormap for ROIs
    n_colors = 20
    cmap = get_cmap(n_colors)

    save_dir = kwargs.get('save_dir', './')
    
    # load all data
    rois        = np.load(roi_path, allow_pickle=True)
    video       = imread(video_path)
    tail_angles = np.genfromtxt(tail_path, delimiter=",")[:, 1:]
    
    # Optionally apply gamma and contrast adjustment
    gamma    = 1
    contrast = 1.5
    video = adjust_gamma(adjust_contrast(video, contrast), gamma)
    
    baseline = np.mean(tail_angles[:100, :])
    
    tail_angles -= baseline

    prefix = os.path.basename(video_path)[:-4]

    temporal_footprints   = rois[()]['roi_temporal_footprints']
    temp_residuals        = rois[()]['roi_temporal_residuals']
    spatial_footprints    = rois[()]['roi_spatial_footprints']
    bg_temp_footprints    = rois[()]['bg_temporal_footprints']
    bg_spatial_footprints = rois[()]['bg_spatial_footprints']
    removed_rois          = rois[()]['removed_rois']

    # Create x arrays
    one_frame    = fps_tail/fps_calcium # number of tail angle frames in one frame of calcium imaging data (single plane)
    total_frames = int(np.floor(one_frame*(video.shape[0]+frame_offset+1))) # number of tail angle frames in all frames of calcium imaging data (single plane)
    x   = np.linspace(0, video.shape[0], total_frames) # x array in calcium imaging frames
    x_s = np.linspace(0, (video.shape[0]+frame_offset+1)/fps_calcium, total_frames) # x array in seconds
    
    # Create the figure
    fig = plt.figure(figsize=(30, 25), dpi=200)
    
    # Create gridspecs
    gs0 = gridspec.GridSpec(1, 4, width_ratios=[1, 2, 2, 1])
    gs1 = gridspec.GridSpecFromSubplotSpec(int(video.shape[1]/2)*3+2, 2, width_ratios=[0.9, 0.05], height_ratios=[1, 0.1]+[0.1, 0.8, 0.1]*int(video.shape[1]/2), subplot_spec=gs0[0])
    gs2 = gridspec.GridSpecFromSubplotSpec(int(video.shape[1]/2)*3+2, 2, width_ratios=[0.1, 1.9], height_ratios=[1, 0.1]+[0.1, 0.8, 0.1]*int(video.shape[1]/2), subplot_spec=gs0[1], wspace=0)
    gs3 = gridspec.GridSpecFromSubplotSpec(int(video.shape[1]/2)*3+2, 2, width_ratios=[0.1, 1.9], height_ratios=[1, 0.1]+[0.1, 0.8, 0.1]*int(video.shape[1]/2), subplot_spec=gs0[2], wspace=0)
    gs4 = gridspec.GridSpecFromSubplotSpec(int(video.shape[1]/2)*3+2, 2, width_ratios=[0.05, 0.9], height_ratios=[1, 0.1]+[0.1, 0.8, 0.1]*int(video.shape[1]/2), subplot_spec=gs0[3])
    
    # -- Z-SCORE COLORBAR PLOT -- #
    ax = plt.subplot(gs4[0:2, 0])
    fig.add_subplot(ax)
    
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    cb1 = matplotlib.colorbar.ColorbarBase(plt.gca(), cmap=colormap,
                                norm=norm,
                                orientation='vertical')
    plt.ylabel('Z-score')
    
    # -- TAIL BEAT FREQUENCY COLORBAR PLOT -- #
    tail_beat_frequency = calculate_tail_beat_frequency(fps_tail, tail_angles[:total_frames, -1])

    ax = plt.subplot(gs1[0:2, 1])
    fig.add_subplot(ax)
    
    norm = matplotlib.colors.Normalize(vmin=np.amin(tail_beat_frequency), vmax=np.amax(tail_beat_frequency))
    cb1 = matplotlib.colorbar.ColorbarBase(plt.gca(), cmap=colormap,
                                norm=norm,
                                orientation='vertical')
    ax.yaxis.set_label_position("left")
    ax.yaxis.set_ticks_position('left')
    plt.ylabel('Tail Beat Frequency')
    
    for j in range(2):
        # -- TAIL ANGLE PLOT -- #
        if j == 0:
            ax = plt.subplot(gs2[0, 1])
        else:
            ax = plt.subplot(gs3[0, 1])
        fig.add_subplot(ax)
        
        plt.plot(x_s, tail_angles[:total_frames, -1], c='b', lw=0.5)
        plt.xlim(0, (video.shape[0]+frame_offset+1)/fps_calcium)
        plt.ylabel('Tail Angle (Degrees)')
        plt.xlabel('Time (s)')
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().spines['right'].set_visible(False)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['bottom'].set_visible(False)
        plt.gca().yaxis.set_ticks_position('left')
        plt.ylim(-180, 180)
        plt.yticks([-180, -90, 0, 90, 180])
        
        # -- TAIL BEAT FREQUENCY PLOT -- #
        if j == 0:
            ax = plt.subplot(gs2[1, 1])
        else:
            ax = plt.subplot(gs3[1, 1])
        fig.add_subplot(ax)
        
        plt.imshow(tail_beat_frequency[np.newaxis, :], aspect='auto', cmap=colormap, extent=[0, (video.shape[0]+frame_offset+1)/fps_calcium, 0, 1])
        plt.title('Tail Beat Frequency (Hz)')
        plt.xlabel('Time (s)')
        plt.gca().get_yaxis().set_visible(False)
        plt.gca().spines['right'].set_visible(False)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['left'].set_visible(False)
        plt.gca().margins(2)
    
    # Copy the temporal footprints list
    temp_footprints = temporal_footprints[:]
    for z in range(video.shape[1]):
        all_colors = [ (np.random.uniform(50, 200), np.random.uniform(50, 200), np.random.uniform(50, 200)) for i in range(10000) ]
        
        # Make a list of ROIs to plot (just the kept ROIs)
        kept_rois = [ i for i in range(temporal_footprints[z].shape[0]) if i not in removed_rois[z] ]
        
        # Compute z-scored data
        zscore = (temporal_footprints[z][kept_rois] - np.mean(temporal_footprints[z][kept_rois], axis=1)[:, np.newaxis])/np.std(temporal_footprints[z][kept_rois], axis=1)[:, np.newaxis]
        
        # Compute sorted data based on correlations of z-scored data
        correlations = np.corrcoef(zscore)
        i, j = np.unravel_index(correlations.argmin(), correlations.shape)

        temp_footprints[z] = temporal_footprints[z][kept_rois]
        temp_footprints[z][0] = zscore[i]
        temp_footprints[z][-1] = zscore[j]
        
        sorted_kept_rois = kept_rois[:]
        sorted_kept_rois[0] = kept_rois[i]
        sorted_kept_rois[-1] = kept_rois[j]

        remaining_indices = [ index for index in range(temp_footprints[z].shape[0]) if index not in (i, j) ]
        for k in range(1, temp_footprints[z].shape[0]-1):
            corrs_1 = [ correlations[i, index] for index in remaining_indices ]
            corrs_2 = [ correlations[j, index] for index in remaining_indices ]

            difference = [ corrs_1[l] - corrs_2[l] for l in range(len(remaining_indices)) ]
            l = np.argmax(difference)
            index = remaining_indices[l]

            temp_footprints[z][k] = zscore[index]
            
            sorted_kept_rois[k] = kept_rois[index]

            del remaining_indices[l]
        
        # Create the background image (mean of the video)
        video_max = np.amax(video)
        image = np.mean(video[:, z, :, :], axis=0)
        image = 255.0*image/video_max
        image[image > 255] = 255
        image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_GRAY2RGB)
        
        kept_footprints = temp_footprints[z]
        
        # Create the ROI overlays
        roi_spatial_footprints = spatial_footprints[z].toarray().reshape((video.shape[2], video.shape[3], spatial_footprints[z].shape[-1])).transpose((0, 1, 2))
        overlays = np.zeros((roi_spatial_footprints.shape[-1], image.shape[0], image.shape[1], 4)).astype(np.uint8)
        total_mask = np.zeros((image.shape[0], image.shape[1]))
        
        roi_contours = []

        for j in range(len(kept_rois)):
            i = kept_rois[j]
            maximum = np.amax(roi_spatial_footprints[:, :, i])
            mask = (roi_spatial_footprints[:, :, i] > 0).copy()

            contours = cv2.findContours(mask.astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2]

            contour = max(contours, key=cv2.contourArea)
            
            roi_contours.append(contour)

            M = cv2.moments(contour)
            if M["m00"] != 0:
                center_x = int(M["m10"] / M["m00"])
                center_y = int(M["m01"] / M["m00"])
            else:
                center_x = 0
                center_y = 0
            
            color = cmap(j % n_colors)[:3]
            color = [255*color[0], 255*color[1], 255*color[2]]
    
            overlay = np.zeros((image.shape[0], image.shape[1], 4)).astype(np.uint8)

            overlay[mask, :-1] = color
            overlay[mask, -1] = 205.0*roi_spatial_footprints[mask, i]/maximum + 50
            overlays[i] = overlay
            
            total_mask += mask
        
        # Create the final ROI image (blending the mean image and the overlays)
        denominator = np.count_nonzero(overlays[kept_rois], axis=0)
        denominator[denominator == 0] = 1
        roi_overlay = (np.sum(overlays[kept_rois], axis=0)/denominator).astype(np.uint8)
        
        image = blend_transparent(image, roi_overlay)
    
        # -- ROI SPATIAL FOOTPRINTS PLOT -- #
        if z < video.shape[1]/2:
            ax = plt.subplot(gs1[z*3+2:z*3+5, 0:2])
        else:
            ax = plt.subplot(gs4[(z-5)*3+2:(z-5)*3+5, 0:2])
        fig.add_subplot(ax)
        
        plt.imshow(image)
        
        # Add text label for each ROI
        for j in range(len(kept_rois)):
            i = kept_rois[j]
            x = np.amax(roi_contours[j][:, 0, 0])
            y = np.amax(roi_contours[j][:, 0, 1])
            
            color = cmap(j % n_colors)[:3]
            
            plt.text(x, y, i, color=color)
        
        plt.axis('off')
        
        # -- ROI IDENTIFICATION PLOT -- #
        if z < video.shape[1]/2:
            ax = plt.subplot(gs2[z*3+3, 0])
        else:
            ax = plt.subplot(gs3[(z-5)*3+3, 0])
        fig.add_subplot(ax)
        
        # Create image to plot next to ROI traces which identifies each ROI's color and number
        arr = np.zeros((len(kept_rois), 1, 3)).astype(np.uint8)
        for i in range(len(kept_rois)):
            arr[i, :, :] = all_colors[kept_rois[i]]
            
            color = cmap(i % n_colors)[:3]
            color = [255*color[0], 255*color[1], 255*color[2]]
            arr[i, :, :] = color
        plt.imshow(arr, aspect='auto', extent=[0, 1, 0, len(kept_rois)])
        
        # Add text label for each ROI
        for i in range(len(kept_rois)):
            plt.text(0, i, i, size=9)
        
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().spines['right'].set_visible(False)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['bottom'].set_visible(False)
        plt.gca().yaxis.set_ticks_position('left')
        plt.ylabel('ROI #')
        plt.yticks([0, kept_footprints.shape[0]])
        
        # -- ROI TEMPORAL TRACES PLOT -- #
        if z < video.shape[1]/2:
            ax = plt.subplot(gs2[z*3+3, 1])
        else:
            ax = plt.subplot(gs3[(z-5)*3+3, 1])
        
        fig.add_subplot(ax)
        
        # Plot z-scored traces sorted using correlations
        heatmap = (kept_footprints - np.mean(kept_footprints, axis=1)[:, np.newaxis])/np.std(kept_footprints, axis=1)[:, np.newaxis]
        plt.imshow(heatmap, aspect='auto', cmap=colormap, extent=[(frame_offset + z/video.shape[1])/fps_calcium, (frame_offset + temp_footprints[z].shape[1]+z/video.shape[1])/fps_calcium, 0, temp_footprints[z].shape[0]], vmin=vmin, vmax=vmax)

        plt.gca().get_yaxis().set_visible(False)
        plt.xlabel('Time (s)')
        plt.gca().spines['right'].set_visible(False)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['left'].set_visible(False)
        plt.ylim(0, kept_footprints.shape[0])
        plt.xlim(0, (video.shape[0]+frame_offset+1)/fps_calcium)
    
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(save_dir + "{}_zscore_{}_to_{}.png".format(prefix, vmin, vmax))
    plt.savefig(save_dir + "{}_zscore_{}_to_{}.svg".format(prefix, vmin, vmax))
    
    # (Optional) Show the plot
    # plt.show()

    plt.cla()
    plt.clf()

In [9]:
main_path  = 'E:/CaImAn/Calcium-Imaging-Analysis/Fiji files/Feb.26.19_huc6f__fish2_6dpf_PAG_lv40_2A/'
video_path = main_path+"Feb.26.19_huc6f__fish2_6dpf_PAG_lv40_2A.tif"
roi_path   = main_path+"roi_data.npy"
tail_path  = "E:/TailTracking/Feb.26.19_2A/Feb.26.19_2A_tail_angles1.csv"

# the important part - run the function to create the plot
create_plot(video_path, roi_path, tail_path, save_dir=main_path)

FileNotFoundError: [Errno 2] No such file or directory: 'E:/CaImAn/Calcium-Imaging-Analysis/Fiji files/Feb.26.19_huc6f__fish2_6dpf_PAG_lv40_2A/roi_data.npy'

In [17]:
video_path = "/Volumes/Things/Tod/Videos from Michael/Aug.22.18_HUCGC6f_7dpf_fish3_prey_10dps_1px_3B_mc.tif"
roi_path   = "/Volumes/Things/Tod/Videos from Michael/Filtered/Aug.22.18_HUCGC6f_7dpf_fish3_prey_10dps_1px_3B_mc/roi_data.npy"
tail_path  = "/Volumes/Things/Tod/Videos from Michael/Tail Tracking/Aug.22.18_3B_tail_angles.csv"

# the important part - run the function to create the plot
create_plot(video_path, roi_path, tail_path, save_dir="/Users/jordan/Downloads/")

<Figure size 6000x5000 with 0 Axes>